
# 프로젝트 목적

> **북한 관련 뉴스 기사**로부터 위치 정보를 자동 추출하고,    
> **지리 좌표(위도·경도)** 및 행정구역 정보(시·도, 시·군·구)와 **매핑**하여    
> **지도 시각화**가 가능한 형태로 정제하는 것을 목표로 합니다.    

---

## 📂 전체 파이프라인 구성 요약

```bash
/final_preprocessing.json                          ← 원본 기사
├── 1.geo_extractor.py                             ← spaCy + 사전 기반 위치 추출기
├── 2.geo_extractor_result_get_id_loc.py           ← 기사 ID와 location 리스트만 정리
├── 3.geo_output_merge.ipynb                       ← ID 기준 메타데이터 병합
├── 4.geocoding_location_mapping.ipynb             ← 위경도 매핑 + 행정구역 보완
└── Dictionary_data/                               ← 북한 지역/시설 단어장 (JSON)
```

---

## 단계별 설명

### ① 위치 정보 추출 (NER + 사전 기반 탐색)

* **파일**: `1.geo_extractor.py`
* **입력**: `/final_preprocessing.json`
* **딕셔너리 경로**: `Dictiionary_data/` 내부 JSON
* **출력**: `extracted_locations_ten_year_all.jsonl`

**기술 요약**:

* `spaCy`의 한국어 모델(`ko_core_news_lg`)을 이용해 `GPE`, `LOC` 엔티티를 추출
* 별도 구축된 북한 관련 지명·시설명 단어장(JSON)을 기반으로 사전 매칭
* `"지역명 + 시설명"` 형태는 정규식을 통해 **복합 개체**로 추출
* 문장 내 북한 맥락(`북한`, `김정은`, `DPRK`) 존재 여부를 기반으로 **신뢰도(high/medium)** 부여
* 기사당 relevance score(0–100)를 계산해 **북한 관련성 판단**

**사용 모델**:

* `spaCy` 한국어 NER (`ko_core_news_lg` 우선, 대체 옵션 있음)

**딕셔너리 기반 추출 항목**:

* `dprk_region.json` (지역)
* `dprk_military.json`, `dprk_landmark.json`, `dprk_politics.json`, `dprk_edu_cul.json` 등 시설명



---

### ② ID와 location만 필터링 저장

* **파일**: `2.geo_extractor_result_get_id_loc.py`
* **입력**: `extracted_locations_ten_year_all.jsonl`
* **출력**: `extracted_locations_result_ten_year_all.jsonl`

**기능 요약**:

* 기사별 전체 분석 결과 중 `"id_"`와 `"locations"` 필드만 JSONL 형식으로 정리
* 지도 시각화 등에 활용하기 위한 **경량 버전 정제**



---

### ③ 메타데이터 병합 (제목, 날짜, URL 등)

* **파일**: `3.geo_output_merge.ipynb`
* **입력**: 위에서 생성된 `id`, `locations` 파일
* **출력**:

  * `Geo_Merge_data`: `id`, `title`, `pubDate`, `url`, `locations`
  * `combined_data_by_year`: 기사 날짜 기준으로 **연도별 파일 분리**

**기능 요약**:

* ID 기준으로 원본 기사 메타정보와 `locations`를 병합
* 연도 단위로 분리하여 추후 연도별 지도 시각화에 활용 가능

---

### ④ 위경도 매핑 및 행정구역 보완

* **파일**: `4.geocoding_location_mapping.ipynb`
* **입력**: `combined_data_by_year`
* **지오데이터 경로**: `Geosjon_data/` 내 북한 지역 polygon/point 정보
* **출력**:

  * `combined_data_by_year_with_coordinates`: location → point/polygon 좌표 연결 결과
  * `combined_data_by_year_with_coordinates_region_plus`: geopy 포함관계 기반 **시도/시군구 정보 보완까지 완료된 최종본**

**기술 요약**:

* 1단계: GeoJSON 파일에서 이름 기반 매칭 → `geometry` (point/polygon) 추출
* 2단계: `geopy` 라이브러리를 사용하여 **Point → Polygon 포함 여부 확인**

  * ex) 위경도는 있으나 시군구 정보 누락된 경우 → polygon에 포함되면 추론 보완

---

## 전체 로직 요약 (한 줄씩 요약)

| 단계  | 역할                     | 핵심 도구                                |
| --- | ---------------------- | ------------------------------------ |
| 1단계 | 북한 관련 위치 정보 추출         | `spaCy` + 사용자 정의 사전 + 정규식            |
| 2단계 | `id`와 `location`만 필터링  | `json.loads()`                       |
| 3단계 | 원문 메타정보 병합 및 연도별 분할    | `pandas`, `datetime`                 |
| 4단계 | 위경도 좌표 연결 및 행정구역 추정 보완 | `GeoJSON`, `geopy`, point/polygon 매핑 |

---

## 최종 산출물 설명

* **최종 파일 경로**:
  `/home/ds4_sia_nolb/code_from_8_19/sh/geocoding/combined_data_by_year_with_coordinates_region_plus`

* **포맷**: JSONL or DataFrame

* **포함 필드**:

  * `id_`, `title`, `pubDate`, `url`, `locations`
  * `geometry_type`: `Point` / `Polygon`
  * `coordinates`: \[경도, 위도]
  * `region`: 시도명
  * `sub_region`: 시군구명

